In [ ]:
import pandas as pd
import sqlalchemy
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
from sqlalchemy.exc import SQLAlchemyError
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

from db_queries import username, password, dsn, dbhostname, service_name, dbtables, querys
from table_functions import *
from analyze_visualisation import *

In [ ]:
data = {}

try:
    sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
    engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
    for table, query in zip(dbtables, querys):
        data.update({table: pd.read_sql(query, engine)})
except SQLAlchemyError as e:
    print(e)

data = drop_unused_columns(data)

In [ ]:
final_table = combine_final_table(data)
final_table = create_final_status(final_table)
final_table = drop_columns_not_used_in_ml(final_table)

In [ ]:
final_table, categorized_columns = categorize_data(final_table)
save_df_to_csv(final_table, 'final_table_before_standarization.csv')
final_table = standarize_data(final_table)
save_df_to_csv(final_table, 'final_table_before_normalization.csv')
final_table = normalize_data(final_table, categorized_columns)
save_df_to_csv(final_table, 'final_table.csv')

In [ ]:
final_table = read_csv('final_table.csv')

In [ ]:
analyze_data(final_table)

In [ ]:
ml_data = split_data(final_table)

In [ ]:
#describe_our_data(final_table)